In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline  

import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data.dataset import Dataset
from torchvision.models import resnet50, ResNet50_Weights

import torchattacks

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


val_dataset = datasets.ImageFolder(root='./data/Imagenet/val/',
                                    transform=test_transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128,
                                          shuffle=False, num_workers=8)

In [4]:
model_paths = { 'ViT': './models/road_sign/vit_b_16_0.ckpt',
            'DenseNet121':'./models/road_sign/densenet121_0.ckpt',
            'Efficientnet_b2': './models/road_sign/efficientnet_b2_0.ckpt',
            'Resnet18': './models/road_sign/resnet18_0.ckpt',
            'Swin_s': './models/road_sign/swin_s_0.ckpt'}
                    

models = {'ViT_b_16': 'vit_b_16', 'ViT_l_16': 'vit_l_16',
          'DenseNet121': 'densenet121', 'DenseNet161': 'densenet161',
          'Efficientnet_b2':'efficientnet_b2', 'Efficientnet_b1': 'efficientnet_b1',
          'Resnet18': 'resnet18', 'Resnet34': 'resnet34',
          'Swin_s': 'swin_s', 'Swin_t': 'swin_t'}

model_names = ['ViT_b_16', 'DenseNet121', 'Efficientnet_b2', 'Resnet18', 'Swin_s']


eps = 1/255
alpha = 0.25/255
iters = 7
attack = 'Momentum'
save_path = 'save_path_with_a_lot_of_memory'

In [5]:
class model_with_normalization(nn.Module):
    def __init__(self, model, normalization):
        super(model_with_normalization, self).__init__()
        self.model = model 
        self.normalization = normalization
        
    def forward(self, x):
        x = self.normalization(x)
        out = self.model(x)
        return out
    
normalization = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])

In [6]:
# for model_name in model_names:
#     m = torch.hub.load("pytorch/vision", models[model_name],
#                            weights="IMAGENET1K_V1").to(device)
#     m.eval()
#     m = model_with_normalization(m, normalization)
#     # atk = torchattacks.PGD(m,random_start=False, eps=eps,alpha=alpha,steps=iters)
#     atk = torchattacks.MIFGSM(m, eps=eps, steps=iters, decay=0.9)
#     # atk = torchattacks.FGSM(m, eps=eps)
    
#     true_labels = []
#     adv_inputs = []
#     sample_indexes = []
#     sample_num = 0
    
#     for batch, labels in val_loader:
#         batch, labels = batch.to(device), labels.to(device)
#         sample_index = torch.arange(sample_num, sample_num+labels.size(0))
#         adv_samples = atk(batch, labels)
#         pred_labels = m(adv_samples).argmax(dim=1)
#         mask = pred_labels.eq(labels)
#         if mask.all():
#             continue
#         else:
#             adv_samples = adv_samples[~mask]
#             labels = labels[~mask]
#             sample_index = sample_index[~mask.cpu()]

#             true_labels.append(labels.to('cpu'))
#             adv_inputs.append(adv_samples.to('cpu'))
#             sample_indexes.append(sample_index)
#         sample_num += labels.size(0)        
        
#     true_labels = torch.cat(true_labels)
#     adv_inputs = torch.cat(adv_inputs, dim=0)
#     sample_indexes = torch.cat(sample_indexes)
    
#     torch.save(true_labels, os.path.join(save_path,
#                                          f'adv_samples/{attack}/imagenet/{model_name}_labels.ckpt'))
#     torch.save(adv_inputs,  os.path.join(save_path,
#                                          f'adv_samples/{attack}/imagenet/{model_name}_samples.ckpt'))
#     torch.save(sample_indexes,  os.path.join(save_path,
#                                              f'adv_samples/{attack}/imagenet/{model_name}_index.ckpt'))
#     del atk, m
     

In [7]:
# m = torch.hub.load("pytorch/vision", models['Swin_s'],
#                        weights="IMAGENET1K_V1")
# m = modify_model_output_classes(m, 3)

In [8]:
# m.load_state_dict(torch.load(model_paths['Swin_s'])['net'])

In [9]:
# weight = m.fc.weight[:3, :]
# bias = m.fc.bias[:3]

In [10]:
# m.fc = nn.Linear(512, 3)
# m.fc.weight.data = weight
# m.fc.bias.data = bias

In [11]:
# state = {'net': m.state_dict()}
# torch.save(state, './models/xray/resnet18_0.ckpt')

In [12]:
other_list = {'Resnet18': ['ViT_b_16','Resnet34', 'DenseNet121', 'Efficientnet_b2', 'Swin_s'],
              'Swin_s': ['ViT_b_16', 'Swin_t', 'DenseNet121', 'Efficientnet_b2', 'Resnet18'],
              'ViT_b_16': ['DenseNet121', 'ViT_l_16', 'Efficientnet_b2', 'Resnet18', 'Swin_s'],
              'DenseNet121': ['ViT_b_16', 'DenseNet161', 'Efficientnet_b2', 'Resnet18', 'Swin_s'],
              'Efficientnet_b2': ['ViT_b_16', 'Efficientnet_b1', 'DenseNet121', 'Resnet18', 'Swin_s'],            
             }

In [13]:
# for surogate, others in other_list.items():
    
#     true_labels= torch.load(os.path.join(save_path,
#                                          f'adv_samples/{attack}/imagenet/{surogate}_labels.ckpt'))
#     adv_inputs = torch.load(os.path.join(save_path,
#                                          f'adv_samples/{attack}/imagenet/{surogate}_samples.ckpt'))
#     sample_index = torch.load(os.path.join(save_path,
#                                              f'adv_samples/{attack}/imagenet/{surogate}_index.ckpt'))
    
#     adv_dataset = torch.utils.data.TensorDataset(adv_inputs, true_labels, sample_index)
#     adv_loader = torch.utils.data.DataLoader(adv_dataset, batch_size=64,
#                                              shuffle=False, num_workers=4)
#     print(f'Testing model {surogate}')
#     results_df = {}
#     for test_model_name in others:
        
#         results_df[f'{test_model_name}_pred_label'] = []
#         results_df[f'{test_model_name}_pred_conf'] = []
#         results_df[f'{test_model_name}_true_conf'] = []
#         results_df[f'{test_model_name}_true_label'] = []
        
#         model = models[test_model_name]
        
#         m = torch.hub.load("pytorch/vision", model,
#                                weights="IMAGENET1K_V1").to(device)
#         m.eval()
#         m = model_with_normalization(m, normalization)

#         true_labels = []
#         adv_inputs = []
#         with torch.no_grad():
#             for batch, labels, _ in adv_loader:
#                 batch = batch.to(device)
#                 outputs = torch.softmax(m(batch), dim=1).cpu()
#                 max_return = outputs.max(dim=1)
                
#                 results_df[f'{test_model_name}_pred_label'].extend(max_return.indices.numpy().tolist())
#                 results_df[f'{test_model_name}_pred_conf'].extend(max_return.values.numpy().tolist())
                
#                 true_conf = torch.gather(outputs, dim=1, index=labels.view(-1, 1)).view(-1)
#                 results_df[f'{test_model_name}_true_conf'].extend(true_conf.cpu().numpy().tolist())
#                 results_df[f'{test_model_name}_true_label'].extend(labels.numpy().tolist())

#         del m
        
#     results_df['sample_index'] = []
#     for _, _, index in adv_loader:
#         results_df['sample_index'].extend(index.numpy().tolist())
        
#     results_df = pd.DataFrame(results_df)
#     results_df.to_csv(f'./results/{attack}/imagenet_surogate_{surogate}.csv', index=False)

In [14]:
# torch.gather(outputs, dim=1, index=labels.view(-1, 1))

In [15]:
# df = {}

# for model_name in model_names:
#     if '_copy' in model_name:
#         continue
#     m = torch.hub.load("pytorch/vision", models[model_name],
#                            weights="IMAGENET1K_V1").to(device)
#     m.eval()
#     m = model_with_normalization(m, normalization)
#     print(f'loaded {model_name}')

#     df[f'{model_name}_probs'] = []
#     df[f'{model_name}_noise_probs'] = []
#     with torch.no_grad():
#         sample_num = 0
#         for batch, labels in val_loader:
#             batch, labels = batch.to(device), labels.to(device)
#             probs = torch.softmax(m(batch), dim=1).max(dim=1).values
#             noise_probs =torch.softmax(m(torch.clamp(batch+
#                                                      (16/255)*torch.randn_like(batch),
#                                                      0, 1)),
#                                        dim=1).max(dim=1).values

#             df[f'{model_name}_probs'].extend(probs.detach().cpu().numpy().tolist())
#             df[f'{model_name}_noise_probs'].extend(noise_probs.detach().cpu().numpy().tolist())
# df['sample_index'] = list(range(len(val_loader.dataset)))        
# df = pd.DataFrame(df)
# df.to_csv(f'./results/{attack}/imagenet_all_preds.csv', index=False)

    
     

In [16]:
import torchvision.transforms as transforms
import cv2

class Resize(object):
    def __init__(self, org_size, inter_size):
        self.org_size = org_size
        self.inter_size = inter_size

    def __call__(self, img):
        img = transforms.functional.resize(img, self.inter_size)
        return transforms.functional.resize(img, self.org_size)
    
class Compress(object):
    def __init__(self,):
        self.encode_param =  [int(cv2.IMWRITE_JPEG_QUALITY), 90]

    def __call__(self, img):
        img = (img.cpu().numpy()*255).astype(np.uint8).transpose([1,2,0])
        result, encimg = cv2.imencode('.jpg', img, self.encode_param)
        decimg = cv2.imdecode(encimg, 1)
        return torch.Tensor(decimg.transpose(2,0,1))/255
    
class Adv_dataset_with_transforms(Dataset):
    def __init__(self, adv_inputs, true_labels, sample_index, transforms):
        self.data = adv_inputs
        self.target = true_labels
        self.sample_index = sample_index
        self.transform = transforms
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index]
        x = self.transform(x)
        
        y = self.target[index]
        sample_index = self.sample_index[index]
        return x, y, sample_index

In [18]:
transform = 'combined'

if transform == 'compression':
    trans = Compress()
elif transform == 'resize':
    trans = Resize(224, 200)
elif transform == 'combined':
    trans = transforms.Compose([
        Compress(),
        Resize(224, 200)
    ])
else:
    trans = lambda x: x

In [19]:
for surogate, others in other_list.items():
    
    true_labels= torch.load(os.path.join(save_path,
                                         f'adv_samples/PGD/imagenet/{surogate}_labels.ckpt'))
    adv_inputs = torch.load(os.path.join(save_path,
                                         f'adv_samples/PGD/imagenet/{surogate}_samples.ckpt'))
    sample_index = torch.load(os.path.join(save_path,
                                             f'adv_samples/PGD/imagenet/{surogate}_index.ckpt'))
    
    adv_dataset = Adv_dataset_with_transforms(adv_inputs, true_labels, sample_index, trans)
    adv_loader = torch.utils.data.DataLoader(adv_dataset, batch_size=64,
                                             shuffle=False, num_workers=4)
    print(f'Testing model {surogate}')
    results_df = {}
    for test_model_name in others:
        
        results_df[f'{test_model_name}_pred_label'] = []
        results_df[f'{test_model_name}_pred_conf'] = []
        results_df[f'{test_model_name}_true_conf'] = []
        results_df[f'{test_model_name}_true_label'] = []
        
        model = models[test_model_name]
        
        m = torch.hub.load("pytorch/vision", model,
                               weights="IMAGENET1K_V1").to(device)
        m.eval()
        m = model_with_normalization(m, normalization)

        true_labels = []
        adv_inputs = []
        with torch.no_grad():
            for batch, labels, _ in adv_loader:
                batch = batch.to(device)
                outputs = torch.softmax(m(batch), dim=1).cpu()
                max_return = outputs.max(dim=1)
                
                results_df[f'{test_model_name}_pred_label'].extend(max_return.indices.numpy().tolist())
                results_df[f'{test_model_name}_pred_conf'].extend(max_return.values.numpy().tolist())
                
                true_conf = torch.gather(outputs, dim=1, index=labels.view(-1, 1)).view(-1)
                results_df[f'{test_model_name}_true_conf'].extend(true_conf.cpu().numpy().tolist())
                results_df[f'{test_model_name}_true_label'].extend(labels.numpy().tolist())

        del m
        
    results_df['sample_index'] = []
    for _, _, index in adv_loader:
        results_df['sample_index'].extend(index.numpy().tolist())
        
    results_df = pd.DataFrame(results_df)
    results_df.to_csv(f'./results/transforms/{transform}/imagenet_surogate_{surogate}.csv', index=False)

Testing model Resnet18


Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main


Testing model Swin_s


Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main


Testing model ViT_b_16


Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main


Testing model DenseNet121


Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main


Testing model Efficientnet_b2


Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main
